<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/01-bow%2Bbinario%2Bsvm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline JCC: BoW+SVM

En esta notebook se presetan los experimentos sobre la estrategia de representación y técnica de aprendizaje *baseline* utilizada para las JCC de  la Universidad Nacional de La Plata.

Para ello vamos a preprocesar los correos y aplicar:
- Bag of words,
- Pesado binario/no binario,
- Máquina de vector soporte (SVM).


## Instalación y Carga de librerías y funciones útiles

### Instalación de librerías

Se instalan las librerías que no están en el entorno de Google Colab

In [1]:
# Se instala gensim que es el que tiene el modelo Word2Vec
!pip install requests
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=ac225817a835892b8913785047b7191a872ae6d3332717693e325f9043dbf4b2
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


### Funciones útiles

Se cargan funciones útiles desde el repo https://github.com/jumafernandez/clasificacion_correos para la carga y balanceo del dataset.

In [2]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/jcc/funciones_dataset.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_dataset.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_dataset import get_clases, cargar_dataset

También se carga la función para preprocesar el texto que se usó en los otros modelos desde el repo: https://github.com/jumafernandez/clasificacion_correos.

In [3]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/jcc/funciones_preprocesamiento.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_preprocesamiento.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_preprocesamiento import preprocesar_correos

### Carga de datos

Se carga el dataframe en memoria con el preprocesamiento de los datos:

In [4]:
import warnings
from os import path
warnings.filterwarnings("ignore")

# Defino la cantidad de clases a utilizar
CANTIDAD_CLASES = 4

# Constantes con los datos
DS_DIR = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/consolidado_jcc/'
TRAIN_FILE = 'correos-train-80.csv'
TEST_FILE = 'correos-test-20.csv'

# Chequeo sobre si los archivos están en el working directory
download_files = not(path.exists(TRAIN_FILE))

etiquetas = get_clases()
train_df, test_df, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

# Se ejecuta el preprocesamiento de correos sobre el campo Consulta de train y test
import pandas as pd
train_df['Consulta'] = pd.Series(preprocesar_correos(train_df['Consulta']))
test_df['Consulta'] = pd.Series(preprocesar_correos(test_df['Consulta']))

# Cambio los integers por las etiquetas
train_df['clase'] = etiquetas[train_df['clase']]
test_df['clase'] = etiquetas[test_df['clase']]

# Las vuelvo a pasar a números 0-N para evitar conflictos con simpletransformers
# Este paso está fijo para estos experimentos
dict_clases_id = {'Otras Consultas': 0,
                            'Ingreso a la Universidad': 1,
                            'Boleto Universitario': 2,
                            'Requisitos de Ingreso': 3}

train_df['clase'].replace(dict_clases_id, inplace=True)
test_df['clase'].replace(dict_clases_id, inplace=True)

# Muestro salida por consola
print('Existen {} clases: {}.'.format(len(train_df.clase.unique()), train_df.clase.unique()))

Se inicia descarga de los datasets.

El conjunto de entrenamiento tiene la dimensión: (800, 24)
El conjunto de testeo tiene la dimensión: (200, 24)
Existen 4 clases: [0 1 2 3].


## SVM

Se carga en memoria la función _grid_search_por_estrategia_representacion_ que va a iterar ajustando los hiperparámetros para las técnica de __SVM__:

In [5]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/jcc/funciones_clasificacion_texto.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_clasificacion_texto.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_clasificacion_texto import gridsearch_por_estrategia_representacion

Se define el espacio de búsqueda para el ajuste de hiperparámetros del modelo:

In [6]:
# Defino una lista con los esquemas de representación
estrategias_representacion = ['BASELINE', 'BOW', 'TFIDF', '3-4-NGRAM-CHARS', '1-2-NGRAM-WORDS']
modelo = 'SVM'

# Defino los parámetros para GridSearchCV
params_svm = {'SVM__C': [0.01, 0.1, 1, 10, 100, 1000], 
              'SVM__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'SVM__class_weight': [None, 'balanced'],
              'SVM__kernel': ['rbf', 'linear', 'poly', 'sigmoid']
              }

Se ejecuta el ajuste de hiperparámetros para cada estrategia de representación en función del espacio de búsqueda:

In [7]:
for estrategia in estrategias_representacion:
  # Llamo a la función que realiza el gridsearch por estrategia  
  gridsearch_por_estrategia_representacion(train_df, test_df, estrategia, modelo, params_svm, 'drive')

Estrategia de representación: BASELINE
Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed: 13.6min finished


Mounted at drive
Estrategia de representación: BASELINE
Parámetros: {'SVM__C': 10, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'rbf', 'clasificador': 'SVM', 'estrategia': 'BASELINE', 'accuracy': 0.85, 'precision': 0.8351234567901235, 'recall': 0.8278361456973765, 'f1_score': 0.83124883351997}
Accuracy Test-Set: 0.85
Estrategia de representación: BOW
Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed: 13.8min finished


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
Estrategia de representación: BOW
Parámetros: {'SVM__C': 10, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'rbf', 'clasificador': 'SVM', 'estrategia': 'BOW', 'accuracy': 0.85, 'precision': 0.8351234567901235, 'recall': 0.8278361456973765, 'f1_score': 0.83124883351997}
Accuracy Test-Set: 0.85
Estrategia de representación: TFIDF
Fitting 5 folds for each of 240 candidates, totalling 1200 fits


...

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
Estrategia de representación: TFIDF
Parámetros: {'SVM__C': 10, 'SVM__class_weight': None, 'SVM__gamma': 0.01, 'SVM__kernel': 'rbf', 'clasificador': 'SVM', 'estrategia': 'TFIDF', 'accuracy': 0.87, 'precision': 0.8646125116713351, 'recall': 0.8295386272763057, 'f1_score': 0.8412555432372505}
Accuracy Test-Set: 0.87
Estrategia de representación: 3-4-NGRAM-CHARS
Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed: 30.7min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 47.9min
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed: 67.7min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed: 70.6min finished


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
Estrategia de representación: 3-4-NGRAM-CHARS
Parámetros: {'SVM__C': 1, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'rbf', 'clasificador': 'SVM', 'estrategia': '3-4-NGRAM-CHARS', 'accuracy': 0.85, 'precision': 0.8529623477297896, 'recall': 0.8233053305434129, 'f1_score': 0.8355941468337398}
Accuracy Test-Set: 0.85
Estrategia de representación: 1-2-NGRAM-WORDS
Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed: 32.1min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 50.3min
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed: 72.0min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed: 75.4min finished


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
Estrategia de representación: 1-2-NGRAM-WORDS
Parámetros: {'SVM__C': 1000, 'SVM__class_weight': None, 'SVM__gamma': 0.0001, 'SVM__kernel': 'rbf', 'clasificador': 'SVM', 'estrategia': '1-2-NGRAM-WORDS', 'accuracy': 0.855, 'precision': 0.8357118983957219, 'recall': 0.824452007672281, 'f1_score': 0.8285959592377169}
Accuracy Test-Set: 0.855


# Referencias
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://medium.com/analytics-vidhya/ml-pipelines-using-scikit-learn-and-gridsearchcv-fe605a7f9e05